In [ ]:
# Refresh modules each time when importing
%load_ext autoreload
%autoreload 2


In [ ]:
from flowFieldWavy import *
import numpy as np
import scipy as sp
from pseudo import chebdif
import matplotlib.pyplot as plt
%matplotlib inline

# Useful scripts for later use 
###(not part of current development)

### Generate numpy arrays for flowfield solutions (velocity, pressure) from Gibson's ChannelFlow 

In [ ]:
f = np.genfromtxt('eq1.asc',dtype=np.complex)
f_p = np.genfromtxt('peq1.asc',dtype=np.complex)
   
x = f[:,3];x_p = f_p[:,3]

# From the database at channelflow.org
Re=4.0e2
a=1.14
b=2.5

nx = 48
nz = 25
L = 24
M = 24
N = 35

# In input file, ordering of modes is different from what I would like it to be.
# It goes, 0,1,..,24,-23,-22,....,-1 in 'x'
# I prefer, -23,-22,..,-1,0,1,...,22,23
# In 'z', it's just 0,..,24 
#     Negative wavenumbers are not included due to symmetry.

# First, getting rid of mode 24 in both x and z 
modes = x.reshape((nx,nz,3,N))
modes = sp.append(modes,x_p.reshape((nx,nz,1,N)),axis=2)
modes = sp.delete(modes,L,0)
modes = sp.delete(modes,M,1)
nx = nx-1 
nz = nz-1
L = L-1
M = M-1

# Now, reordering streamwise wavenumbers
modes = sp.append(modes[L+1:,:,:,:],modes[:L+1,:,:,:],0)
del x,x_p,f,f_p

# Projecting Cheb coefficients onto Cheb collocation nodes
for kx in range(0,nx):
    for kz in range(0,nz):
        for nd in range(0,4):
            modes[kx,kz,nd,:] = chebcoll_vec(modes[kx,kz,nd,:])

# Adding modes for negative spanwise wavenumbers. 
modes = sp.append(modes[::-1,:0:-1].conjugate(),modes,axis=1)
nz = nx

# Deleting pressure from state-vector:
p = modes[:,:,3]
modes = sp.delete(modes,3,2)

vecsize = nx*nz*N
# Finally, reshaping to get a state-vector containing u and w
x0 = modes.reshape(3*vecsize)
p = p.reshape(vecsize)
del modes
#print(modes[L,M,0])

tempDict = getDefaultDict()
L = 3; M = 5; N=31
eps=1.0e-02; gz = 0.8; b= gz/eps; gx= 0.05; a = gx/eps;
tempDict.update({'K':0, 'L':L,'M':M,'N':N, 'eps':eps, 'alpha':a,'beta':b,'nd':4,'isPois':1, 'Re':100.})
x00 = flowFieldWavy(flowDict=tempDict).view4d()
x00[0,L,M,0] = 1.-(x00.y)**2
pF = x00.getScalar(nd=2)
print(x00.shape)
x0 = x00.copy()

### Checking symmetry of a MATLAB solution

In [ ]:
from flowFieldWavy import *
import numpy as np
vF,pF = mapData2ff(eps=0.02,g=1.0,Re=100., theta=0)[:2]
a = vF.flowDict['alpha']
for x in np.arange(0.,np.pi/a,np.pi/a/21.):
    print(np.linalg.norm(vF.ifft(xLoc=x)[0] - vF.ifft(xLoc=x+np.pi/a)[0,::-1]) )
#print(vF.ifft(xLoc=0))

## Solving riblet flow without coordinate transformation

The velocity, $u$, is expanded as

$a_0 + b_0 y + \sum_n [a_n e^{-ny} + b_n e^{ny}] e^{in\beta z}$

For each mode, there are two unknowns, a and b. The boundary conditions are that the velocity is zero at either wall. So, if I set the above expression to zero for some 'm' locations, and use 'm' modes (including zero), I can simply invert the coefficient matrix to obtain all the 'a's and 'b's. 

In [ ]:
import numpy as np

b = 10. # beta
eps = 0.02
n = 21 # Number of positive modes
m = 2*n+1 # Total number of modes including zero



zArr = np.arange(0., 2.*np.pi/b-1.0e-9, 2.*np.pi/b/m)
yShape = 2.*eps*np.cos(b*zArr)
L = np.zeros((2*m, 2*m), dtype=np.complex)

yTop = (1.+yShape).reshape((yShape.size,1))
yBottom = -1.+yShape.reshape((yShape.size,1))
zArr = zArr.reshape((zArr.size,1))

expTopA = -yTop + 1.j*b*zArr
expTopB = yTop + 1.j*b*zArr
expBottomA = -yBottom + 1.j*b*zArr
expBottomB = yBottom + 1.j*b*zArr

nArr = np.arange(-n,n+1).reshape((1,m))

L[:m, :-1:2] = np.exp(nArr*expTopA)       # Factors multiplying a_n, for yTop locations
L[:m, 1::2] = np.exp(nArr*expTopB)       # Factors multiplying b_n, for yTop locations
L[m:, :-1:2] = np.exp(nArr*expBottomA)    # yBottom locations
L[m:, 1::2] = np.exp(nArr*expBottomB)

# For a_0, the factor is 1 = exp(0.) = exp(0.*y + 0.*1.j*b*z), so above assignment works
# For b_0, the factor is y, but the above assignment only assigns a factor of 1
# Correcting factors for b_0
L[:m, 2*n+1:2*n+2] += yTop - 1.
L[m:, 2*n+1:2*n+2] += yBottom - 1.

# To find the factors, a_n, b_n, the null-space of L needs to be found 
#           (since the boundary condition at yTop, yBottom is that velocity is zero)
# Before that, let's see what the rank is
print('Size of L[0]:', L.shape[0])
print('Rank of L is', np.linalg.matrix_rank(L))
np.linalg.null?

## Verifying norm of MATLAB solutions with flowFieldWavy class

The norm isn't going to be quite ~ 10^-15, the reason being that I treat the last Fourier modes are treated differently in my MATLAB solution with regards to truncation in Fourier modes. In flowFieldWavy, I first compute u.ddx(), which already has some truncation in the last modes. In MATLAB, I don't compute u.ddx() separately; I figure out all the triadic interactions in the Fourier space explicitly. 

### The easy way:

In [ ]:
eps = 0.01; g=0.3; theta=0; Re=100.
vf,pf,fnorm,flg = runMATLAB(g=g,eps=eps,theta=theta,Re=Re,N=25,n=3)
vf.residuals(pField=pf)[0].norm()

### The hard way:
As long as the easy way "works", everything's fine. If it stops working, this is what I do. I don't just look at the final residual norm, but also at linear and non-linear terms separately, as well as the difference in the residual vectors.

However, for this to work, I have to fiddle around with my MATLAB code a bit, because the regular code doesn't return the residual vector or the linear and non-linear terms in the residual vector. The default case is to return just the norm of the residual vector along with the state vector. Statements that allow returning the extra vectors must be placed in comments in the MATLAB code. 

And after I run the MATLAB code to obtain all the vectors, I should save the vectors into a .mat file, and copy it into the current path (for Python). I should also set the dictionary parameters properly, because for now, I'm not importing those parameters from MATLAB solutions. 

The script to be run in MATLAB is called 'pyTest' 

In [ ]:
def vecnorm2(vec,N):
    m = vec.size//N
    w = clencurt(N).reshape((1,N))
    W = np.tile(w,(1,m))
    tempVec = vec.reshape(vec.size,1)
    tempVecConj = tempVec.conj().T
    
    vecNorm = np.sqrt(np.abs( np.dot( tempVecConj*W, tempVec ) ))
    return vecNorm[0,0]

In [ ]:
import scipy.io

N = 25; n=5; eps=0.01; Re=100.; 
g = 0.5
a = g/eps; b = 0.
# Read .mat file into numpy
matFile = scipy.io.loadmat('pyTest.mat')

x = np.array(matFile['x'],dtype=np.complex)
resids = np.array(matFile['F'],dtype=np.complex)
L = np.array(matFile['Lterm'],dtype=np.complex)
NL = np.array(matFile['NLterm'],dtype=np.complex)

tempDict = getDefaultDict()
tempDict.update({'K':0,'L':n,'M':0,'N':N,'alpha':a,'beta':b,'Re':Re,'eps':eps,'isPois':1,'nd':4})

residFF = flowFieldWavy(flowDict=tempDict,arr=resids)
xFF = flowFieldWavy(flowDict=tempDict,arr=x)
vFF = xFF.slice(nd=[0,1,2]); pFF = xFF.getScalar(nd=3)
LFF = flowFieldWavy(flowDict=tempDict,arr=L).slice(nd=[0,1,2])
NLFF = flowFieldWavy(flowDict=tempDict,arr=NL).slice(nd=[0,1,2])

In [ ]:
linTerm = pFF.grad() - vFF.laplacian()/Re
linTerm[:,:,:,:,[0,-1]] = vFF[:,:,:,:,[0,-1]]
NLTerm = vFF.convNL()

In [ ]:
print('Error in norm of linear terms (streamwise), starting with mode -n:')
for l in range(n+1):
    print(vecnorm2(LFF[0,l,0,0]-linTerm[0,l,0,0], vFF.N))
print('*************************************')
print('Error in norm of non-linear terms (streamwise), starting with mode -n:')
for l in range(n+1):
    print(vecnorm2(NLFF[0,l,0,0]-NLTerm[0,l,0,0], vFF.N))
    
print('Error in all modes and variables, linear and non-linear respectively:',(LFF-linTerm).norm(), (NLFF-NLTerm).norm())

## Script for flow-vis: theta = 90 (old version)
### Latest implementation is to use flowFieldArray (see next section)

In [ ]:
import numpy as np
from flowField import *
from flowFieldWavy import *

# eps = 10^(-3.+0.025*fileind)   => fileind = 40*(log10(eps) +3)
# g0 = [0.2:0.025:0.6]    => gind = 40*(g-0.2)
# g1 = [0.65:0.05:1.2, 1.3,1.4,1.5]   => gind = 20*(g-0.65)
eps = 0.03/4.;  g = 3.0/2.; Re = 1000
gFileInd = 0
gInd = int(40*(g-0.2))
if g> 0.6:
    gFileInd = 1
    gInd = int(20*(g-0.65))
    if gInd > 11: gInd = int((gInd-11)/2+11)
ReInd = np.int(np.log10(Re)-1)
        
eFileInd = 2*int(40*(np.log10(eps)+3)/2)

folderPath = '/home/sabarish/matData/seprn/'
fileName = 'dataSeprn'+str(gFileInd)+'b90_'+str(eFileInd)+'.mat'

#vfList, pfList,paramDict = data2ff(fName=folderPath+fileName,ind=gInd)
vf, pf,paramDict = data2ff(fName=folderPath+fileName,ind=3*gInd+ReInd)
eps = paramDict['eps']; g = paramDict['g']; Re = paramDict['Re']
printName = 'tecFiles/wavyB90E'+str(-np.log10(eps))+'G'+str(g)+'Re'+str(int(Re))+'.dat'
vf.printPhysicalPlanar(pField=pf, fName=printName)
#gArr = paramDict['g']
#ReArr = paramDict['Re']
#paramDict['g'][3:6]

## Reading data from .mat files into flowFieldArray class instance
Makes it easier to calculate variables, and to plot

1. Reading .mat files from a folder, according to a template, into a Python list
2. Converting that Python list into flowFieldArray instance (class defined in module flowFieldArray.py)

In [ ]:
# Reading all the .mat files takes a while, so this chunk of code needs to be separate from the other stuff
import glob
import os
from flowFieldWavy import *
from flowFieldArray import *
from flowField import *

# These two lines allow iPython to reload modules before executing
%load_ext autoreload
%autoreload 2

theta='0'
homeFolder = '/home/sabarish/Dropbox/gitwork/python3/flowField/'
folderPath = '/home/sabarish/matData/drag/'
fileTemplate = "data_dragT"+theta+"E*.mat" 

os.chdir(folderPath)
fileList = glob.glob(fileTemplate)
for file in fileList[:]:
    if (file[-6:] =='13.mat') or (file[-6:] =='14.mat') or (file[-6:] =='15.mat'):
        #print(file)
        fileList.remove(file)

vListLong =[]; pListLong = []
for file in fileList:
    vList,pList = data2ff(file)[:2]
    vListLong.extend(vList)
    pListLong.extend(pList)
os.chdir(homeFolder)

In [ ]:
# Create flowFieldArray instance out of the list of flowField instances
vArr = flowFieldArray(vListLong,26,24,21)
pArr =  flowFieldArray(pListLong,26,24,21)

print(isinstance(vArr[0,0,0].field, flowField))
# To obtain a 3-d array of, say, volume fluxes of the data in the flowFieldArray:
fluxArr = vArr.getProperty('flux',withBase=True)
#ReTauArr = vArr.getProperty('ReTau')


## Script for printing .dat files for flow-vis (in Tecplot) using flowFieldArray

In [ ]:
eps = 0.2/2.;  g = 0.6/2.; Re = 100
# printing for 2*eps = 0.0045, 0.02, 0.06, 2*g = 0.625, 1.2, 2.0
epsInd = np.argwhere(vArr[:,0,0].getProperty('eps')>=eps)[0,0]
gInd = np.argwhere(vArr[0,:,0].getProperty('g')>=g)[0,0]
ReInd = np.argwhere(vArr[0,0,:].getProperty('Re')>=Re)[0,0]
print(epsInd,gInd,ReInd)
vArrObj = vArr[epsInd, gInd, ReInd]; vf = vArrObj.field; pf = pArr[epsInd,gInd,ReInd].field
eps = vArrObj.eps; g = vArrObj.g; Re = vArrObj.Re

#eps = paramDict['eps']; g = paramDict['g']; Re = paramDict['Re']
printName = 'finalPlots/wavyE'+str(round(-np.log10(eps),2))+'G'+str(round(g,2))+'Re'+str(int(Re))+'.dat'
                                 
vf.printPhysicalPlanar(pField=pf, fName=printName)
print(2*eps,2*g,Re)

## Validation with OpenFOAM, and checking for symmetry

In [ ]:
import numpy as np
from flowFieldWavy import *
from myUtils import *

filePath =  '/home/sabarish/OpenFOAM/sabarish-2.4.x/run/wavyLam/ep02/a15/finest/'
endTime = '400/'
pointsName=filePath+'constant/polyMesh/points'
eps = 0.02
a = 15.
g= a*eps

x,y,z = readPoints(fName=pointsName, nx = 61,ny=501)
yMax =np.max(y)
yMin = np.min(y)
#h = (yMax-yMin)/2.08
h = 0.1
x = x/h; y =(y-h)/h; z=z/h

import matplotlib.pyplot as plt
%matplotlib inline
#vF,pF,paramDict = mapData2ff(g=g, eps=eps, Re=100, theta=0)
vF,pF,fnorm = runMATLAB(g=g,eps=eps,Re=100.,theta=0,N=50,n=5)
#print(paramDict)
print(vF.N, vF.nx)
u = vF.getScalar(nd=0)
v = vF.getScalar(nd=1)
uData = np.zeros(x.shape); vData = np.zeros(x.shape); uUnInt = np.zeros((x.shape[0],x.shape[1],u.N))
for kx in range(x.shape[0]):
    for kz in range(x.shape[1]):
        yBottom = -1.+2.*eps*np.cos(a*x[kx,0,0])
        yT = y[kx,kz]-yBottom-1.
        uData[kx,kz] = u.getPhysical(xLoc=x[kx,0,0], zLoc=z[0,kz,0], yLoc = yT,withBase=True)
        uUnInt[kx,kz] = u.getPhysical(xLoc=x[kx,0,0], zLoc=z[0,kz,0], withBase=True)
        vData[kx,kz] = v.getPhysical(xLoc=x[kx,0,0], zLoc=z[0,kz,0], yLoc = yT)
        
yUnInt = np.zeros((x.shape[0],u.N))
yCheb = chebdif(u.N,1)[0]
for kx in range(x.shape[0]):
    xLoc = x[kx,0,0]
    yUnInt[kx] = yCheb + 2.*u.flowDict['eps']*np.cos(u.flowDict['alpha']*xLoc)
        
#writeFOAMData(vF,pF)
#'''
uFoam,vFoam,wFoam = readFOAMData(fName=filePath+endTime+'U',nx=60,ny=500)
print('max u in uFoam:',np.max(uFoam))
uFoam = uFoam/0.015
vFoam = vFoam/0.015
"""
plt.figure()
plt.title('Error in U')
cLimits = np.arange(0.,0.008,0.0005)
plt.contourf(x[:,0], y[:,0], np.abs(uData[:,0]-uFoam[:,0]),levels=cLimits)
plt.colorbar()
#plt.clim([-0.008,0.008])
plt.savefig('./figures/uDiffG10.pdf', format='pdf')


plt.figure()
plt.title('Relative difference in U (truncated at 2%)')
uRel = np.abs((uData[:,0]-uFoam[:,0])/uFoam[:,0])
uRel[uRel>0.02] = 0.02; uRel[uRel<-0.02] = -0.02
plt.contourf(x[:,0], y[:,0],uRel , 50)
plt.colorbar()
plt.clim([0.,0.02])
plt.savefig('./figures/uRelDiffG10.pdf', format='pdf')

plt.figure()
plt.title('U from OpenFOAM')
plt.contourf(x[:,0], y[:,0], uFoam[:,0], 50)
plt.colorbar()
plt.clim([0.,0.95])
print('max U velocity from foamData:',np.max(uFoam))
plt.savefig('./figures/uFoamG10.pdf', format='pdf')

plt.figure()
plt.title('U from current spectral solver')
plt.contourf(x[:,0], y[:,0], uData[:,0], 50)
plt.colorbar()
plt.clim([0.,1.0])
plt.savefig('./figures/uMatG10.pdf', format='pdf')

plt.figure()
plt.title('OpenFOAM symmetry')
nx = x.shape[0]
u1 = uFoam[:nx//2,0]; u2 = uFoam[nx//2:,0,::-1]
udif = (u2-u1)/u1
udif[udif>0.05]=0.05; udif[udif<-0.05] = -0.05
plt.contourf(x[:nx//2,0], y[:nx//2,0],udif , 50)
plt.colorbar()

plt.figure()
plt.title('MATLAB solution symmetry- Interpolated')
nx = x.shape[0]
u1 = uData[:nx//2,0]; u2 = uData[nx//2:,0,::-1]
udif = (u2-u1)/u1
udif[udif>0.05]=0.05; udif[udif<-0.05] = -0.05
plt.contourf(x[:nx//2,0], y[:nx//2,0],udif , 50)
plt.colorbar()
"""
plt.figure()
plt.title('Flow symmetry:Relative difference in U (truncated at 1%)')
nx = x.shape[0]
u1 = uUnInt[:nx//2,0]; u2 = uUnInt[nx//2:,0,::-1]
#print('a*xLoc/pi for symmetry: ', a*x[:nx//2,0,0]/np.pi)
udif = np.abs((u2-u1)/u1)
udif[udif>0.01]=0.01
plt.contourf(x[:nx//2,0,:u.N], yUnInt[:nx//2],udif , 50)
plt.colorbar()
plt.savefig('./figures/uSymG06.pdf',format='pdf')

"""
plt.figure()
plt.title('Error in V')
plt.contourf(x[:,0], y[:,0], vData[:,0]-vFoam[:,0], 50)
plt.colorbar()
plt.savefig('vDiffG10.pdf',format='pdf')

plt.figure()
plt.title('V from OpenFOAM')
plt.contourf(x[:,0], y[:,0], vFoam[:,0], 50)
plt.colorbar()
plt.savefig('vFoamG10.pdf',format='pdf')

plt.figure()
plt.title('V from current solver')
plt.contourf(x[:,0], y[:,0], vData[:,0], 50)
plt.colorbar()
plt.savefig('vMatG10.pdf',format='pdf')

plt.figure()
plt.title('Relative difference in V (truncated at 2%)')
vRel = np.abs((vData[:,0]-vFoam[:,0])/vFoam[:,0])
vRel[vRel>0.02] = 0.02
plt.contourf(x[:,0], y[:,0],vRel , 50)
plt.colorbar()
plt.savefig('./figures/vRelDiffG10.pdf', format='pdf')

plt.show()

#'''
"""

##Equivalent wall shift

In [ ]:
hEq = np.sqrt(3./2.*uBulk)
epsEq = 1.-hEq

plt.figure()
plt.title('$\\epsilon_{eq}$ at $Re_{\\tau}$='+str(round(np.sqrt(20),1)))
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], epsEq[:,:,0])
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('epsEqRe10.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('$\\epsilon_{eq}$ at $Re_{\\tau}$='+str(round(np.sqrt(200),1)))
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], epsEq[:,:,10])
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('epsEqRe100.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('$\\epsilon_{eq}$ at $Re_{\\tau}$='+str(round(np.sqrt(2000),1)))
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], epsEq[:,:,20])
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('epsEqRe1000.pdf',format='pdf')
plt.show()


## Run a flow case in Matlab and print to tecPlot format for visualization

In [ ]:
g = 0.1; eps=0.01; Re=100.
printName = 'finalPlots/wavyE'+str(round(-np.log10(eps),2))+'G'+str(round(g,2))+'Re'+str(int(Re))+'.dat'
#vF, pF, fNorm,flg = runMATLAB(g=g, eps=eps, theta=0, Re=Re, N=80,n=13, multi=True)
vF, pF, fNorm = mapData2ff(g=g, eps=eps, theta=0, Re=Re)
print('fNorm is ',fNorm)
vF.printPhysicalPlanar(pField=pF, fName=printName,withBase=True)
print(2*eps,2*g,Re)
print('printed to',printName)

In [ ]:
print(flg)

In [ ]:
uBulk = vF.flux(withBase=True)
hEq = np.sqrt(3./2.*uBulk)
epsEq = 1.-hEq
print(epsEq)

In [ ]:
vF1, pF1, fNorm1,flg1 = runMATLAB(g=g, eps=eps, theta=0, Re=Re, N=65,n=7, multi=True)
print('fNorm is ',fNorm)

In [ ]:
vF.div().ifft(withBase=True)

In [ ]:
vF.div().ifft

In [ ]:
from flowFieldWavy import *

g = 5.0; eps=0.1; Re=1000.
printName = 'finalPlots/wavyE'+str(round(-np.log10(eps),2))+'G'+str(round(g,2))+'Re'+str(int(Re))+'.dat'
vF, pF, fNorm,flg = runMATLAB(g=g, eps=eps, theta=0, Re=Re, N=80,n=13, multi=True)
print('fNorm is ',fNorm)
vF.printPhysicalPlanar(pField=pF, fName=printName,withBase=True)
print(2*eps,2*g,Re)
print('printed to',printName)

In [ ]:
uBulk1 = vF.flux(withBase=True)
hEq1 = np.sqrt(1.5*uBulk1)
epsEq1 = 1.-hEq1
print(epsEq1)
print(flg)

In [ ]:
g = 2.0; eps=0.05; Re=1000.
vF1, pF1, fNorm1,flg1 = runMATLAB(g=g, eps=eps, theta=0, Re=Re, N=80,n=13, multi=True)
print('fNorm is ',fNorm1)
uBulk1 = vF1.flux(withBase=True)
hEq1 = np.sqrt(1.5*uBulk1)
epsEq1 = 1.-hEq1
print(epsEq1)
print(flg)

In [ ]:
uProfile1 = vF1.getScalar().ifft(withBase=True)[0,:]
y1,DM1 = chebdif(vF1.N,1)
D1 = DM1[:,:,0]
plt.plot(uProfile1,y1)
plt.plot(np.dot(D1,uProfile1),y1)
plt.show()

In [ ]:
uProfile = vF.getScalar().ifft(withBase=True)[0,:]
print(vF.N)
print(uProfile.shape)
from pseudo import chebdif
import matplotlib.pyplot as plt
%matplotlib inline
y,DM = chebdif(vF.N,1)
D = DM[:,:,0]
plt.plot(uProfile,y)
plt.plot(np.dot(D,uProfile),y)
plt.show()

In [ ]:
from flowFieldWavy import *
from pseudo import chebdif
import matplotlib.pyplot as plt
%matplotlib inline

xArr = np.arange(0.,0.032,0.0032)
vF,pF,fNorm = mapData2ff(eps = 0.01,g = 1.5,Re=1000.,theta=0)
divField = vF.div().getPhysical(xLoc=xArr)[0,:,0].reshape((10,vF.N))


printName = 'finalPlots/wavyE'+str(round(-np.log10(0.01),2))+'G'+str(round(1.0,2))+'Re'+str(int(1000.))+'.dat'
print('fNorm is ',fNorm)
vF.printPhysicalPlanar(pField=pF, fName=printName,withBase=True)
print('printed to',printName)

y = chebdif(vF.N,1)[0]
plt.contourf(y,np.arange(0.,0.032,0.0032),divField)
plt.colorbar()
plt.show()

plt.contourf(y,xArr, vF.getScalar(nd=0).getPhysical(xLoc=xArr,withBase=True)[0,:,0].reshape(10,vF.N))
plt.colorbar();plt.show()

In [ ]:
vF.view4d().flowDict

In [ ]:
div2 = vF.getScalar(nd=0).ddx() + vF.getScalar(nd=1).ddy()
print(vF.getScalar(nd=0).ddX().ifft(xLoc=0.01) )
print(vF.getScalar(nd=0).ddxzYcomp().ifft(xLoc=0.01))
u = vF.getScalar(nd=0)

In [ ]:
dudx = vF.getScalar(nd=0).ddx()
dudx1 = vF.getScalar(nd=0).ddX()
duTrun = dudx.slice(L=2,N=30)
duTrun1 = dudx1.slice(L=2,N=30)

In [ ]:
type(vF)

In [ ]:
duTrun.ifft(xLoc=0.01)-duTrun1.ifft(xLoc=0.01)

## Streamwise averaging of streamwise velocity and its wall-normal derivative

In [ ]:
# Run the first block of code to import modules

# Define a geometry and flow condition
eps = 0.1; g=1.5; Re=1000.; theta=0

# Read flowfield from database
#vF, pF, fNorm = mapData2ff(eps=eps,g=g,Re=Re,theta=theta)
# Or run a case as needed
vF, pF, fNorm , flg = runMATLAB(g=g, eps=eps, theta=0, Re=Re, N=80,n=13, multi=True)


In [ ]:
1-np.sqrt(1.5*vF.flux(withBase=True))

In [ ]:
U = vF.getScalar(nd=0).addBase()
V = vF.getScalar(nd=1)
U_y = U.ddy()
U_yy = U.ddy2()
yCheb = chebdif(vF.N,1)[0]

# We'll do the plotting at these streamwise locations:
lda = 2.*np.pi*eps/g     # Wavelength
xArr = np.arange(0., lda, lda/21.)

# Now, I need to decide on a set of nodes to interpolate U, U_y, and U_yy on.
# First, defining bottom and top surfaces.
yBottom = -1.+ 2.*eps*np.cos(g/eps*xArr)
yTop = 2.+yBottom
ySurf = 2.*eps*np.cos(g/eps*xArr)

# Nodes for interpolation are these: cheb nodes spread between -1+2*eps and 1-2*eps, which is the region between the crests
yInter = (1.+yCheb)*(1.-2.*eps)

# Defining x and y grids
xGrid = np.tile(xArr.reshape(xArr.size,1),(1,vF.N))
yGrid = np.tile((yInter-1.+2.*eps).reshape(1,vF.N),(xArr.size,1))
UGrid = np.zeros(xGrid.shape)
VGrid = UGrid.copy()
U_yGrid = UGrid.copy()
U_yyGrid = UGrid.copy()

# Calculating U, U_y, and U_yy on the above grids. flowField methods ifft() and getPhysical reference 'y' from the bottom wall
#     I need to account for this
for k in range(xArr.size):
    print(xArr[k],ySurf[k])
    UGrid[k] = U.getPhysical(xLoc=xArr[k], yLoc = yInter-ySurf[k]+2.*eps-1.)[0,0,0]
    VGrid[k] = V.getPhysical(xLoc=xArr[k], yLoc = yInter-ySurf[k]+2.*eps-1.)[0,0,0]
    U_yGrid[k] = U_y.getPhysical(xLoc=xArr[k], yLoc = yInter-ySurf[k]+2.*eps-1.)[0,0,0]
    U_yyGrid[k] = U_yy.getPhysical(xLoc=xArr[k], yLoc = yInter-ySurf[k]+2.*eps-1.)[0,0,0]

plt.contourf(xGrid,yGrid,UGrid,20)
plt.title('U');plt.colorbar();plt.show()

plt.figure()
plt.contourf(xGrid,yGrid,VGrid,20)
plt.title('V');plt.colorbar();plt.show()

plt.figure()
plt.contourf(xGrid,yGrid,U_yGrid,levels=np.arange(-2.,2.1,0.1))
plt.title('U_y');plt.colorbar();plt.clim([-2.,2.]);plt.show()

plt.figure()
plt.contourf(xGrid,yGrid,U_yyGrid,levels = np.arange(-2.5,-1.4,0.1))
plt.title('U_yy');plt.colorbar();plt.show()

## Continued in next block of code

In [ ]:
# Plot U_y at a few streamwise locations
plt.figure()
for k in np.arange(0,xArr.size):
    plt.plot(yGrid[k],U_yGrid[k])
plt.title('U_y');plt.xlabel('y');plt.ylim([-3.,3.]);plt.ylabel('U_y');plt.show()

plt.figure()
# The linear U_y profile would be U_y = -2y
for k in np.arange(0,xArr.size):
    plt.plot(yGrid[k],U_yGrid[k]+2.*yGrid[k])
plt.title('U_y+2*y');plt.xlabel('y');plt.ylim([-3.,3.]);plt.ylabel('U_y');plt.show()

U_yMean = np.mean(U_yGrid,axis=0)

plt.figure()
plt.plot(yInter-0.8, U_yMean)
plt.title('Streamwise averaged U_y'); plt.xlabel('y'); plt.ylabel('U_y'); plt.xlim(-0.8,0.8); plt.show()

plt.figure()
plt.plot(yInter-0.8, U_yMean+2.*(yInter-0.8))
plt.title('Streamwise averaged U_y + 2*y'); plt.xlabel('y'); plt.ylabel('U_y'); plt.xlim(-0.8,0.8); plt.show()

UMean = np.mean(UGrid,axis=0)
plt.figure()
plt.plot(yInter-0.8, (UMean-1.+(yInter-0.8)**2))
plt.title('Streamwise averaged U - (1-y**2)'); plt.xlabel('y'); plt.ylabel('U'); plt.xlim(-0.8,0.8);  plt.show()

In [ ]:
1.-(1.5*vF.flux(withBase=True))**(1./3.)

In [ ]:
Ub = vF.getScalar(nd=0).addBase().intY().ifft()[0,0]/1.6
print(1.-np.sqrt(1.5*Ub))

## Scripts for Plotting (clean them up later)

In [ ]:
#from seprn import seprn
#%load_ext autoreload
#%autoreload 2

xs = np.zeros(vArr.shape,dtype=np.float)
xr = xs.copy(); ybub=xs.copy(); sflag = np.zeros(vArr.shape,dtype=np.int)

for ke in range(xs.shape[0]):
    for kg in range(xs.shape[1]):
        for kr in range(xs.shape[2]):
            x = vArr[ke,kg,kr].field.addBase().appendField(pArr[ke,kg,kr].field).view1d()
            N = x.N; n = x.flowDict['L']; a = x.flowDict['alpha']; b=x.flowDict['beta'];Re=x.flowDict['Re']; eps=x.flowDict['eps']
            x = x.copyArray()
            sf,sp,rp,yb = seprn(x,N,n,eps,a,b,Re)
            xs[ke,kg,kr] = sp; xr[ke,kg,kr] = rp; ybub[ke,kg,kr] = yb; sflag[ke,kg,kr] = sf


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
epsArr = vArr[:,:,0].getProperty('eps')
gArr = vArr[:,:,0].getProperty('g')

plt.figure()
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,10])
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=4.5$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,0],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT4.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=8$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,5],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT8.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=14.1$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,10],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT14.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=45$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,20],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT45.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=4.5$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,0],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT4.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=8$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,5],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT8.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=14.1$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,10],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT14.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=45$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,20],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT45.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=4.5$')
plt.contourf(2.*gArr,2.*epsArr,ybub[:,:,0],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT4.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=8$')
plt.contourf(2.*gArr,2.*epsArr,ybub[:,:,5],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT8.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=14.1$')
plt.contourf(2.*gArr,2.*epsArr,ybub[:,:,10],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT14.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=45$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,20],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT45.pdf',format='pdf')
plt.show()


In [ ]:
#fluxArr = vArr.getProperty('flux',withBase=True)

import matplotlib.lines as mlines
eOff = 7
plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=4.5$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,0],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,0],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT4.pdf',format='pdf')
plt.show()



plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=8$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,5],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,5],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT8.pdf',format='pdf')
plt.show()



plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=14$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,10],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,10],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT14.pdf',format='pdf')
plt.show()



plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=45$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,20],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,20],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT45.pdf',format='pdf')
plt.show()





In [ ]:
#ReTauArr = vArr.getProperty('ReTau')
ReArr = vArr.getProperty('Re')
#fluxArr = vArr.getProperty('flux',withBase=True)
uBulk = fluxArr; ReFlat = ReArr
print(ReFlat.shape, uBulk.shape, type(ReFlat), type(uBulk))
fricFac = 16./ReFlat/uBulk/uBulk
ReWavy = uBulk*ReFlat
epsArr = vArr.getProperty('eps'); gArr = vArr.getProperty('g')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
phiArr = ReWavy*fricFac
keArr = np.asarray([6,17,21]);kgArr = np.asarray([1,11,22]); nRe=ReWavy.shape[2]
print(epsArr[keArr,0,0])
print(gArr[0,kgArr,0])

markers = ['*','s','^']

kg=kgArr[0]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])
plt.xlim([-100,700])

ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
plt.savefig('./figures/phiRe0.pdf',format='pdf')

ax1=ax0.scatter(ReWavy[keArr[0],kg,:], phiArr[keArr[0],kg,:], marker=markers[0],color='b',s=40)
plt.savefig('./figures/phiRe11.pdf',format='pdf')

ax2=ax0.scatter(ReWavy[keArr[1],kg,:], phiArr[keArr[1],kg,:], marker=markers[1],color='c',s=40)
plt.savefig('./figures/phiRe12.pdf',format='pdf')

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe13.pdf',format='pdf')


kg=kgArr[1]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])

ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
plt.savefig('./figures/phiRe0.pdf',format='pdf')

ax1=ax0.scatter(ReWavy[keArr[0],kg,:], phiArr[keArr[0],kg,:], marker=markers[0],color='b',s=40)
plt.savefig('./figures/phiRe21.pdf',format='pdf')

ax2=ax0.scatter(ReWavy[keArr[1],kg,:], phiArr[keArr[1],kg,:], marker=markers[1],color='c',s=40)
plt.savefig('./figures/phiRe22.pdf',format='pdf')

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe23.pdf',format='pdf')


kg=kgArr[2]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])

ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
#plt.savefig('./figures/phiRe0.pdf',format='pdf')

ax1=ax0.scatter(ReWavy[keArr[0],kg,:], phiArr[keArr[0],kg,:], marker=markers[0],color='b',s=40)
plt.savefig('./figures/phiRe31.pdf',format='pdf')

ax2=ax0.scatter(ReWavy[keArr[1],kg,:], phiArr[keArr[1],kg,:], marker=markers[1],color='c',s=40)
plt.savefig('./figures/phiRe32.pdf',format='pdf')

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe33.pdf',format='pdf')

plt.show()

kg = kgArr[0]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r')
phiMean = 0.5*(np.min(phiArr[keArr[2],kg,:])+np.max(phiArr[keArr[2],kg,:]))
phiDev = np.max(phiArr[keArr[2],kg,:]) - phiMean
print('phiMean, phiDev:',phiMean,phiDev)
ax4=ax0.plot(ReWavy[keArr[2],kg,:],phiMean*np.ones(nRe), color='r')


plt.annotate('$\\bar{\\phi}$',xy = (0.05,0.5),xycoords='axes fraction', 
             xytext=(0.05,0.5),textcoords='axes fraction',
             arrowprops=dict(facecolor='white',shrink=0.000,width=0.0),
            horizontalalignment='left', verticalalignment='top',fontsize=16)

plt.show()



In [ ]:
kg=kgArr[0]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])
ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe40.pdf',format='pdf')
plt.show()

fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe41.pdf',format='pdf')

phiMean = 0.5*(np.min(phiArr[keArr[2],kg,:])+np.max(phiArr[keArr[2],kg,:]))
phiDev = np.max(phiArr[keArr[2],kg,:]) - phiMean
print('phiMean, phiDev:',phiMean,phiDev)
ax4=ax0.plot(ReWavy[keArr[2],kg,:],phiMean*np.ones(nRe), color='r')
plt.text(-35,25.6,'$\\bar{\\phi}$',fontsize=35,color='blue')
plt.savefig('./figures/phiRe42.pdf',format='pdf')

plt.annotate("",xy=(600,26.22),xycoords='data',
            xytext=(600,25.58),textcoords='data',
             arrowprops={'arrowstyle':'<->'})
plt.text(530,25.9,"$\\phi'$",fontsize=35,color='blue')
plt.savefig('./figures/phiRe43.pdf',format='pdf')
plt.show()


In [ ]:
phiMean = 0.5*(np.min(phiArr,axis=2)+np.max(phiArr,axis=2))
phiDev = 0.5*(np.max(phiArr,axis=2)-np.min(phiArr,axis=2))

## Curve fitting

- We're looking for a relation that goes as $\lambda = a Re^{-n}$, where $a$ and $n$ are functions of $g,\epsilon$. 
- From earlier work by Adrien, we know $n \approx 1$ for $\epsilon$ holds for $4\epsilon$ not exceeding 15%.
- We just need to verify if $\lambda Re = const.$ holds up, and if it does, how we need to see how the constant varies with $g,\epsilon$.

### FIrst, checking if the inverse law holds

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
#coloring = np.random.random(32).reshape((8,4))
#for k in range(0,8):
#    plt.scatter(np.log10(ReWavy[3*k,3*k]),10.**(np.log10(ReWavy[3*k,3*k])+ np.log10(fricFac[3*k,3*k])), c=coloring[k-3])
#plt.ylim([-0.006,0.])
#plt.show()

#theta='0'
#pp = PdfPages('InverseLaw'+theta+'.pdf')

#prod = fricFac*ReWavy         # This must be a constant for different ReWavy. 
prod = phiArr
# To see if it is indeed constant, I can look at the standard deviation at any g,eps:
#prod = prod[:,:,2:]

#stdDev = np.std(prod,axis=2)  # Computing standard deviation for lambda*Re at a constant g,eps, but varying Re
stdDev=phiDev

epsArr = vArr.getProperty('eps')
gArr = vArr.getProperty('g')
cutOff = 0

cLimits = np.arange(24., 44., 1.0)
#meanProd = np.mean(prod,axis=2)  # Computing mean for lambda*Re at a constant g,eps, but varying Re
meanProd = phiMean
plt.figure()
#plt.contourf(2.*gArr[:-cutOff,:,0], 4.*epsArr[:-cutOff,:,0], meanProd[:-cutOff],levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], meanProd[:], levels=cLimits)
#plt.ylim([24.,44.])
#plt.clim(24.0,28.0)
plt.colorbar()
plt.title('$\\bar{\\phi}$',fontsize=25)
plt.xlabel('Slope',fontsize=15);plt.ylabel('Amplitude',fontsize=15)
#pp.savefig()
plt.savefig('./figures/mean.pdf',format='pdf')
plt.show()

cLimits= np.arange(0.0, 0.3, 0.015)
plt.figure()
plt.title("$\\phi'$",fontsize=25)
#plt.contourf(2.*gArr[:-cutOff,:,0], 4.*epsArr[:-cutOff,:,0], stdDev[:-cutOff], levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:], levels=cLimits)
#plt.clim([0.,0.15])
#plt.clim(0.0,0.2)
plt.colorbar()
plt.xlabel('Slope',fontsize=15);plt.ylabel('Amplitude',fontsize=15)
#pp.savefig()
plt.savefig('./figures/std.pdf',format='pdf')
plt.show()


cLimits = np.arange(0.0,0.1,0.1/20)
#cLimits= np.append(cLimits,np.asarray([0.2,0.3,0.4,0.5,0.6,0.7]))
plt.figure()
plt.title("$\\phi'/\\bar{\\phi}$",fontsize=25)
#plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]-24),levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]),levels=cLimits)
#plt.clim([0.,0.15])
#plt.clim(0.0,0.1)
plt.colorbar()
plt.xlabel('Slope',fontsize=15);plt.ylabel('Amplitude',fontsize=15)
plt.savefig('./figures/stdOverMean.pdf',format='pdf')
plt.show()
#plt.savefig('stdOverMeanT'+theta+'.pdf')

# Mean where stdDev/(mean-24) is small enough (<10%):
#cLimits = np.arange(0., 4., 0.25)
#stdOverMean = stdDev/(meanProd-24.)
stdOverMean = stdDev/(meanProd)
newMean = meanProd.copy()
newMean[stdOverMean>0.1] = None
plt.figure()
#plt.title('(Mean-24) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.title('(Mean) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], (newMean[:]-24))
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('meanInverseLaw'+theta+'.pdf',format='pdf')
plt.show()




In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
#coloring = np.random.random(32).reshape((8,4))
#for k in range(0,8):
#    plt.scatter(np.log10(ReWavy[3*k,3*k]),10.**(np.log10(ReWavy[3*k,3*k])+ np.log10(fricFac[3*k,3*k])), c=coloring[k-3])
#plt.ylim([-0.006,0.])
#plt.show()

#theta='0'
#pp = PdfPages('InverseLaw'+theta+'.pdf')

prod = fricFac*ReWavy         # This must be a constant for different ReWavy. 
# To see if it is indeed constant, I can look at the standard deviation at any g,eps:
#prod = prod[:,:,2:]
#meanProd = np.mean(prod,axis=2)  # Computing mean for lambda*Re at a constant g,eps, but varying Re
meanProd = 0.5*( prod[:,:,0] + prod[:,:,-1] )

#stdDev = np.std(prod,axis=2)  # Computing standard deviation for lambda*Re at a constant g,eps, but varying Re
stdDev = np.amax( np.abs( prod[:,:,:] -meanProd.reshape((prod.shape[0],prod.shape[1],1))  ) , axis=2 )

epsArr = vArr.getProperty('eps')
gArr = vArr.getProperty('g')
cutOff = 0

cLimits = np.arange(24., 44., 1.0)
plt.figure()
#plt.contourf(2.*gArr[:-cutOff,:,0], 4.*epsArr[:-cutOff,:,0], meanProd[:-cutOff],levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], meanProd[:], levels=cLimits)
#plt.ylim([24.,44.])
#plt.clim(24.0,28.0)
plt.colorbar()
plt.title('$\\bar{\\phi}$: Mean of $\\phi(Re)$')
plt.xlabel('Slope');plt.ylabel('Amplitude')
#pp.savefig()
plt.savefig('mean.pdf',format='pdf')
plt.show()

#cLimits= np.arange(0.0, 0.3, 0.05)
plt.figure()
plt.title("$\\phi'$: Deviation of $\\phi(Re)$ from $\\bar{\\phi}$")
#plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:], levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:],20)
#plt.clim([0.,0.15])
#plt.clim(0.0,2.0)
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
#pp.savefig()
plt.savefig('stdT'+theta+'.pdf',format='pdf')
plt.show()


cLimits = np.arange(0.0,0.15,0.1/20)
#cLimits= np.append(cLimits,np.asarray([0.2,0.3,0.4,0.5,0.6,0.7]))
plt.figure()
plt.title("$\\phi'/\\bar{\\phi}$: Relative deviation")
#plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]-24),levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]),levels=cLimits)
#plt.clim([0.,0.15])
#plt.clim(0.0,0.1)
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('stdOverMeanT'+theta+'.pdf',format='pdf')
plt.show()
#plt.savefig('stdOverMeanT'+theta+'.pdf')

# Mean where stdDev/(mean-24) is small enough (<10%):
#cLimits = np.arange(0., 4., 0.25)
#stdOverMean = stdDev/(meanProd-24.)
stdOverMean = stdDev/(meanProd)
newMean = meanProd.copy()
newMean[stdOverMean>0.1] = None
plt.figure()
#plt.title('(Mean-24) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.title('(Mean) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], (newMean[:]-24))
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('meanInverseLaw'+theta+'.pdf',format='pdf')
plt.show()


In [ ]:
plt.figure()
lArr = np.random.rand(120).reshape(30,4)
symArr = ["o","v","s","^","*","x","+"]
symArr.extend(symArr);symArr.extend(symArr);
printFolder = '/home/sabarish/Dropbox/reports/2015/transfer/figures/'
'''
for k in range(21):
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    plt.scatter(2.*gArr[k,:,0],(meanProd[k,:]-24.), color=lArr[k])
plt.show()
'''
plt.figure()
#ax = plt.subplot(111)
#ax = plt.axes()
plt.annotate('Slope',xy = (0.4,0.7),xycoords='axes fraction', 
             xytext=(0.7,0.1),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
keArr=[0,1,2,3,4,6,8,11,14,18,23]
for k in keArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*epsArr[:,k,0],(meanProd[:,k]-24.), color=lArr[k])
    plt.plot(2.*epsArr[:,k,0],(meanProd[:,k]))#, marker=symArr[k])
plt.title('$\\bar{\\phi}$-dependence on Amplitude')
plt.xlabel('Amplitude')
plt.ylabel('$\\bar{\\phi}$',fontsize=12)
plt.ylim([23.,43.])
plt.savefig('./figures/epsDep.pdf',format='pdf')
plt.show()

kgArr=[0,5,10,14,17,19,21,22,23,24,25]
plt.figure()
plt.annotate('Amplitude',xy = (0.5,0.9),xycoords='axes fraction', 
             xytext=(0.5,0.05),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
for k in kgArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*gArr[k,4:,0],(meanProd[k,4:]-24.), color=lArr[k])
    plt.plot(2.*gArr[k,4:,0],(meanProd[k,4:]))
plt.title('$\\bar{\\phi}$-dependence on Slope')
plt.xlabel('Slope')
plt.ylabel('$\\bar{\\phi}$',fontsize=12)
plt.ylim([23.,43.])
plt.savefig('./figures/gDep.pdf',format='pdf')
plt.show()




plt.figure()
nRe = ReWavy[0,0,:].size

#keArr = np.asarray([3,10,20,3,10,20,3,10,20]);kgArr = np.asarray([0,0,0,10,10,10,20,20,20])
"""
for k in range(keArr.size):
    ke = keArr[k]; kg = kgArr[k]
    plt.figure()
    ax1=plt.scatter(ReWavy[ke,kg,:], ReWavy[ke,kg,:]*fricFac[ke,kg,:])
    ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax3,=plt.plot(ReWavy[ke,kg,:], 24.*np.ones(nRe),linestyle='--')
    plt.legend([ax1,ax2,ax3], ['computed', 'mean','flatWall'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('$\bar{\phi}$')
    plt.ylim([23.,29.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.title('$\bar{\phi}$ for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')
"""
keArr = np.asarray([3,11,20]);kgArr = np.asarray([0,10,20])
markers = ['x','+','^']
for k in range(kgArr.size):
    kg = kgArr[k]
    fig = plt.figure(figsize=(7,5))
    ax0 = fig.add_subplot(111)
    ax1=ax0.scatter(ReWavy[keArr[0],kg,:], ReWavy[keArr[0],kg,:]*fricFac[keArr[0],kg,:], marker=markers[0],color='b')
    ax2=ax0.scatter(ReWavy[keArr[1],kg,:], ReWavy[keArr[1],kg,:]*fricFac[keArr[1],kg,:], marker=markers[1],color='g')
    ax3=ax0.scatter(ReWavy[keArr[2],kg,:], ReWavy[keArr[2],kg,:]*fricFac[keArr[2],kg,:], marker=markers[2],color='r')
    #ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
    plt.legend([ax1,ax2,ax3,ax4], \
               ['Amp:'+str(round(2.*epsArr[keArr[0],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[1],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[2],0,0],4)),\
                'flat'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('phi')
    plt.ylim([23.,30.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.title('$\\bar{\\phi}$ for Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')




In [ ]:
plt.figure()
plt.annotate('Slope',xy = (0.4,0.7),xycoords='axes fraction', 
             xytext=(0.7,0.1),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
keArr=[0,1,2,3,4,6,8,11,14,18,23]
for k in keArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*epsArr[:,k,0],(meanProd[:,k]-24.), color=lArr[k])
    plt.plot(2.*epsArr[:,k,0],(meanProd[:,k]))#, marker=symArr[k])
plt.title('$\\bar{\\phi}$-dependence on Amplitude',fontsize=18)
plt.xlabel('Amplitude',fontsize=15)
plt.ylabel('$\\bar{\\phi}$',fontsize=25)
plt.ylim([23.,43.])
plt.savefig('./figures/epsDep.pdf',format='pdf')
plt.show()

kgArr=[0,5,10,14,17,19,21,22,23,24,25]
plt.figure()
plt.annotate('Amplitude',xy = (0.5,0.9),xycoords='axes fraction', 
             xytext=(0.5,0.05),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
for k in kgArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*gArr[k,4:,0],(meanProd[k,4:]-24.), color=lArr[k])
    plt.plot(2.*gArr[k,4:,0],(meanProd[k,4:]))
plt.title('$\\bar{\\phi}$-dependence on Slope',fontsize=18)
plt.xlabel('Slope',fontsize=15)
plt.ylabel('$\\bar{\\phi}$',fontsize=25)
plt.ylim([23.,43.])
plt.savefig('./figures/gDep.pdf',format='pdf')
plt.show()


In [ ]:
plt.figure()
lArr = np.random.rand(120).reshape(30,4)
symArr = ["o","v","s","^","*","x","+"]
symArr.extend(symArr);symArr.extend(symArr);
printFolder = '/home/sabarish/Dropbox/reports/2015/transfer/figures/'
'''
for k in range(21):
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    plt.scatter(2.*gArr[k,:,0],(meanProd[k,:]-24.), color=lArr[k])
plt.show()
'''
plt.figure()
#ax = plt.subplot(111)
#ax = plt.axes()
plt.annotate('Slope',xy = (0.4,0.7),xycoords='axes fraction', 
             xytext=(0.7,0.1),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
keArr=[0,1,2,3,4,6,8,11,14,18,23]
for k in keArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*epsArr[:,k,0],(meanProd[:,k]-24.), color=lArr[k])
    plt.plot(2.*epsArr[:,k,0],(meanProd[:,k]))#, marker=symArr[k])
#plt.title('Amplitude depencence of $\\bar{\\phi}$')
plt.xlabel('Amplitude ($2 \\epsilon$)')
plt.ylabel('$\\bar{\\phi}$')
plt.ylim([23.,43.])
plt.savefig(printFolder+'epsDep.pdf',format='pdf')
plt.show()

kgArr=[0,5,10,14,17,19,21,22,23,24,25]
plt.figure()
plt.annotate('Amplitude',xy = (0.5,0.9),xycoords='axes fraction', 
             xytext=(0.5,0.05),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
for k in kgArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*gArr[k,4:,0],(meanProd[k,4:]-24.), color=lArr[k])
    plt.plot(2.*gArr[k,4:,0],(meanProd[k,4:]))
#plt.title('Slope dependence of $\\bar{\\phi}$')
plt.xlabel('Slope (2g)')
plt.ylabel('$\\bar{\\phi}$')
plt.ylim([23.,43.])
plt.savefig(printFolder+'gDep.pdf',format='pdf')
plt.show()




plt.figure()
nRe = ReWavy[0,0,:].size

#keArr = np.asarray([3,10,20,3,10,20,3,10,20]);kgArr = np.asarray([0,0,0,10,10,10,20,20,20])
"""
for k in range(keArr.size):
    ke = keArr[k]; kg = kgArr[k]
    plt.figure()
    ax1=plt.scatter(ReWavy[ke,kg,:], ReWavy[ke,kg,:]*fricFac[ke,kg,:])
    ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax3,=plt.plot(ReWavy[ke,kg,:], 24.*np.ones(nRe),linestyle='--')
    plt.legend([ax1,ax2,ax3], ['computed', 'mean','flatWall'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('$\bar{\phi}$')
    plt.ylim([23.,29.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.title('$\bar{\phi}$ for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')
"""
keArr = np.asarray([3,11,20]);kgArr = np.asarray([0,10,20])
markers = ['x','+','^']
for k in range(kgArr.size):
    kg = kgArr[k]
    fig = plt.figure(figsize=(7,5))
    ax0 = fig.add_subplot(111)
    ax1=ax0.scatter(ReWavy[keArr[0],kg,:], ReWavy[keArr[0],kg,:]*fricFac[keArr[0],kg,:], marker=markers[0],color='b')
    ax2=ax0.scatter(ReWavy[keArr[1],kg,:], ReWavy[keArr[1],kg,:]*fricFac[keArr[1],kg,:], marker=markers[1],color='g')
    ax3=ax0.scatter(ReWavy[keArr[2],kg,:], ReWavy[keArr[2],kg,:]*fricFac[keArr[2],kg,:], marker=markers[2],color='r')
    #ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
    plt.legend([ax1,ax2,ax3,ax4], \
               ['Amp:'+str(round(2.*epsArr[keArr[0],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[1],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[2],0,0],4)),\
                'flat'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('$\\phi$')
    plt.ylim([23.,30.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    #plt.title('Re-variation of $\\phi$, for Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')




In [ ]:
from scipy.optimize import leastsq

meanData = meanProd[:, 4:]
epsVec2 = 2.*epsArr[:,0,0]; gVec2 = 2.*gArr[0,4:,0]
eps2d = 2.*epsArr[:,4:,0]; g2d = 2.*gArr[:,4:,0]



eps1d = eps2d.reshape(eps2d.size); g1d = g2d.reshape(g2d.size)
meanData1d = meanData.reshape(meanData.size)

def __residuals(params,meanData1d,eps1d, g1d):
    # Fitting meanData to  (a*tanh(b*g2d)+c)*eps2d +d*eps2d
    # a = params[0]; b=params[1]; c=params[2]
    #residual = 24+(params[0]*np.tanh(params[1]*g1d) + params[2])*eps1d + params[3]*eps1d**2  - meanData1d
    
    # Fitting meanData to 24 + [A*tanh(B*g1d) + 72-A]*eps1d + 144.*eps1d 
    residual = 24+(params[0]*np.tanh(params[1]*g1d) + 72.-params[0])*eps1d + (params[2]*np.tanh(params[3]*g1d) + 144.-params[2])*eps1d**2  - meanData1d
    return residual

#print(meanData.shape, eps2d.shape, g2d.shape)
#test = __residuals([70.,1.,10.], meanData, eps2d, g2d)
#print(type(test),test.shape)
pSol = leastsq(__residuals, np.array([50.,1.,150.,1.]), args=(meanData1d,eps1d,g1d))[0]

plt.figure()
plt.contourf(g2d, eps2d,  meanData, levels=np.arange(23.,43.,1.))
plt.colorbar()
plt.title('$\\bar{\\phi}$')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
#plt.savefig('mean.pdf',format='pdf')
#plt.yscale('log')
plt.show()

plt.figure()
plt.contourf(g2d, eps2d, __residuals(pSol, meanData,eps2d,g2d)+meanData, levels=np.arange(23.,43.,1.0))
plt.colorbar()
plt.title('Model: $(A\\tanh(Bg) +72-A)\epsilon + (C\\tanh(Dg) +144-C)\epsilon^2$')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
#plt.yscale('log')
plt.savefig('./figures/model.pdf', format='pdf')
plt.show()

plt.figure()
plt.contourf(g2d, eps2d, np.abs(__residuals(pSol, meanData,eps2d,g2d)), levels=np.arange(0.,0.5,0.01))
plt.colorbar()
plt.title('Error in $\\bar{\\phi}$')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
#plt.yscale('log')
plt.savefig('./figures/modelDiff.pdf', format='pdf')
plt.show()

plt.figure()
plt.contourf(g2d, eps2d, np.abs(__residuals(pSol, meanData,eps2d,g2d)/meanData), levels=np.arange(0.,0.02,0.001))
plt.colorbar()
plt.title('Relative error in $\\bar{\\phi}$',fontsize=20)
plt.xlabel('Slope',fontsize=15); plt.ylabel('Amplitude',fontsize=15)
#plt.yscale('log')
plt.savefig('./figures/modelRelDiff.pdf', format='pdf')
plt.show()

print('Model coefficients:a=%1.3f,b=%1.3f,c=%1.3f,d=%1.3f'%(pSol[0],pSol[1],pSol[2], pSol[3]) )



In [ ]:
# Printing to ascii for 3-d visualization in tec360:
fricFac2d = fricFac[:,:]; ReWavy2d = ReWavy[:,:]
ReWavy1d = ReWavy2d.reshape(ReWavy2d.size); fricFac1d = fricFac2d.reshape(fricFac2d.size)
eps2d = 2.*epsArr; g2d = 2.*gArr
eps1d = eps2d.reshape(eps2d.size); g1d = g2d.reshape(g2d.size)
modelMean = ((pSol[0]*np.tanh(pSol[1]*g1d) + pSol[2])*eps1d + pSol[3]*eps1d**2)+24.
modelFricFac = modelMean/ReWavy1d
#modelFricFac = ((.*np.tanh(0.6*g1d) + 10.)*eps1d + 136.*eps1d**2)+24.
fricFacDiff = np.abs(modelFricFac-fricFac1d)
fricFacRelDiff = np.abs(modelFricFac-fricFac1d)/fricFac1d

modelDiff = np.abs(modelFricFac - fricFac1d*ReWavy1d)
modelDiffRelative = modelDiff/(fricFac1d*ReWavy1d-24.)

printArr = np.concatenate((ReWavy1d, g1d, eps1d, fricFac1d, modelFricFac, fricFacDiff, fricFacRelDiff), axis=0)
fName='./figures/modelFFv1'
variables = 'VARIABLES = "Re(bulk)", "Slope", "Amplitude", "fricFac","fricFacModel", "fricFacError", "fricFacRelError"\n'
zone = 'ZONE T="", I='+str(ReWavy2d.shape[2])+', J='+str(ReWavy2d.shape[1])+', K='+str(ReWavy2d.shape[0])+', DATAPACKING=POINT \n'
title = 'TITLE= "Product of friction factor and Reynolds number: computed and modeled"\n'
hdr = title+variables+zone

print(type(printArr),printArr.shape)
np.savetxt(fName+'.dat',printArr.reshape(7,printArr.size//7).T, header=hdr,comments='')
print('Printed data to file %s.dat'%fName)



In [ ]:
epsVec = epsArr[:-5,0,0]
plt.plot(epsVec,aArr)
plt.title('eps vs a: in mean = a*tanh(b*g) +c')
plt.xlabel('eps'); plt.ylabel('a');plt.show()


plt.figure()
plt.plot(epsVec,bArr)
plt.title('eps vs b in mean = a*tanh(b*g) +c')
plt.xlabel('eps'); plt.ylabel('b');plt.show()

plt.figure()
plt.plot(epsVec,cArr)
plt.title('eps vs c in mean = a*tanh(b*g) +c')
plt.xlabel('eps'); plt.ylabel('c');plt.show()


In [ ]:
linFun = lambda xData,k: k*xData
sqrtFun = lambda xData,k: k*np.sqrt(xData)

aCoeff,pcov = curve_fit(linFun,epsVec, aArr, p0=100.)
bCoeff,pcov = curve_fit(sqrtFun, epsVec,bArr)
cCoeff,pcov = curve_fit(linFun, epsVec, cArr,p0=20.)

print(aCoeff, bCoeff,cCoeff)

In [ ]:
epsArr2 = epsArr[:,:,0]; gArr2 = gArr[:,:,0]
curveFitted = 24.+ cCoeff*epsArr2 + aCoeff*epsArr2*np.tanh(bCoeff*np.sqrt(epsArr2)*2.*gArr2)
originalData = meanProd[:-5]

plt.contourf(2.*gArr2, 4.*epsArr2,originalData-24.)
plt.title('Original data (mean-24)')
plt.xlabel('slope=2*g'); plt.ylabel('crest-trough amplitude = 4*eps')
plt.colorbar(); plt.show()


plt.figure()
plt.contourf(2.*gArr2, 4.*epsArr2, curveFitted-24.)
plt.title('Curve fitted data (mean-24)')
plt.xlabel('slope=2*g'); plt.ylabel('crest-trough amplitude = 4*eps')
plt.colorbar(); plt.show()

plt.figure()
plt.contourf(2.*gArr2, 2.*epsArr2, np.abs(curveFitted-originalData))
plt.title('Error')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
plt.colorbar(); plt.show()

plt.figure()
plt.contourf(2.*gArr2, 2.*epsArr2, np.abs(curveFitted-originalData)/originalData, levels = np.arange(0.,0.4,0.04))
plt.clim([0.,0.2])
plt.xlabel('Slope'); plt.ylabel('Amplitude')
plt.title('Relative Error')
plt.colorbar(); plt.show()
